## US Hit Streaming Songs Project - Data Collection
By Jeremias Figueroa

Ever since 10 years old, I was fascinated by music charts such as the Billboard Hot 100. Like many Gen-Z, I grew up with streaming services, and I became fascinated by how music changed as a result of the on-demand streaming era. As a result of this passion, I collected and continue to collect week-to-week numbers since 2017 on the weekly on-demand audio streams top hit songs recieve in the United States. Specifically, I collect data from the HITS Daily Double magazine on this Google Sheet: https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit?usp=sharing. 

This project aims to analyze the characteristics of hit songs and what styles of music does the general public prefer to listen to. It utilizes various Python packages such as gspread to connect Python with Google Sheets and spotipy which hosts Spotify's API data.

The following Notebook collects data from Google Sheets and updates the Google Sheet's data utilizing Google and Spotify's API. 

## Imports

First, install the following (add "--upgrade" to upgrade the package).
These packages are to be used in the following notebooks as well.

pip install spotipy

pip install pandas

pip install gspread

pip install oauth2client 

pip install df2gspread

pip install matplotlib

pip install seaborn

pip install sklearn

In [1]:
#to access spotipy
import configparser
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

#to allow interaction with underlying operating system
import os

#to parse through spotipy's output
import json

#for data analysis
import numpy as np
import pandas as pd

#to remove the limits on a dataframe's columns displayed
pd.set_option('display.max_columns', None)

#to access google sheets with python
import gspread
from google.oauth2 import service_account
from oauth2client.service_account import ServiceAccountCredentials

#to edit google sheets from python
from df2gspread import df2gspread as d2g

#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Spotify API Client Credentials

First, ensure that you have a Spotify For Developers account. Then, using https://developer.spotify.com/dashboard/applications, make an new app and get client ID and client secret ID. 

It is recommendable practice to hide passwords from code. Therefore, environmental variables are to be created.

##  CONVERT THE FOLLOWING TO CODE

#The following creates environmental variables

%env SPOTIPY_CLIENT_ID=YOUR_SPOTIFY_APP_CLIENT_ID

%env SPOTIPY_CLIENT_SECRET=YOUR_SPOTIFY_APP_CLIENT_SECRET_ID

%env SPOTIPY_REDIRECT_URI=http://127.0.0.1:8080/
        

#output is hidden/cell is converted to "markdown" to protect privacy

In [3]:
#import spotify client credentials 
spotify_client_id = os.environ['SPOTIPY_CLIENT_ID']
spotify_secret = os.environ['SPOTIPY_CLIENT_SECRET']
spotify_redirect_uri = os.environ['SPOTIPY_REDIRECT_URI']

#create a Spotify instance
client_credentials_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

#to confirm 
print(sp)

## Importing the Google Sheet

Use song information from the following Google Sheet: https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit?usp=sharing

This Google Sheet is a collection of hit songs from HITS Daily Double, a music industry trade publication, and their weekly Overall Song Streams chart. The data includes how many times a song was streamed week-to-week for most weeks since 2017 (with the exception of holiday weeks, weeks when holiday music regularly push out non-holiday songs out of the top spots of music charts). 

In order to be able to use this data on your own device, create a copy of the Google Sheet to your own Drive. Also, follow the instructions in the following link to connect Google Drive with Python: https://github.com/PrettyPrinted/youtube_video_code/tree/master/2021/10/14/How%20to%20Use%20Google%20Sheets%20With%20Python%20(2021)

For this python notebook, the file for our service account is the json file with the Google API keys downloaded from Google Developers Console. Please reference the link for more instructions.

*** TO-DO: CONVERT THE GSPREAD IMPORT TO A FUNCTION (since it's done repeatedly in this project)

In [4]:
#service account
sa = gspread.service_account(filename="gspread_service_account.json")

#sheet
sh = sa.open("HITS Streaming Songs")

#worksheet
wks = sh.worksheet("HITS Streaming Songs")

#uses all values in the worksheet for the data frame
gsheets_df = pd.DataFrame(wks.get_all_values())

header = gsheets_df.iloc[0] #isolate first row as header
gsheets_df = gsheets_df[1:] #get rid of header in original df
gsheets_df.columns = header

#set 'index' as index
gsheets_df = gsheets_df.set_index('index')

gsheets_df['title'] = gsheets_df['title'].str.rstrip(' ')
gsheets_df['artist'] = gsheets_df['artist'].str.rstrip(' ')
gsheets_df['album'] = gsheets_df['album'].str.rstrip(' ')

#convert index from string to int
gsheets_df.index = gsheets_df.index.astype(int)

#fill in blank values with 0
gsheets_df = gsheets_df.replace(r'^\s*$', "0", regex=True)

#convert all numbers-as-strings to numbers from row 4 in python's index and onwards
for col_name in gsheets_df.columns.to_list()[4:len(gsheets_df.columns.to_list())]:
    
    #get rid of strings' commas and convert strings to integers
    gsheets_df[col_name] = gsheets_df[col_name].str.replace(',','').astype('int')
    
#turn peak_date strings to date format
gsheets_df['peak_date'] = pd.to_datetime(gsheets_df['peak_date'])

gsheets_df

,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,peak_week_streams,current_year,2022-06-17,2022-06-10,2022-06-03,2022-05-27,2022-05-20,2022-05-13,2022-05-06,2022-04-29,2022-04-22,2022-04-15,2022-04-08,2022-04-01,2022-03-25,2022-03-18,2022-03-11,2022-03-04,2022-02-25,2022-02-18,2022-02-11,2022-02-04,2022-01-28,2022-01-21,2022-01-14,2022-01-07,2021-12-31,2021-12-24,2021-12-17,2021-12-10,2021-12-03,2021-11-26,2021-11-19,2021-11-12,2021-11-05,2021-10-29,2021-10-22,2021-10-15,2021-10-08,2021-10-01,2021-09-24,2021-09-17,2021-09-10,2021-09-03,2021-08-27,2021-08-20,2021-08-13,2021-08-06,2021-07-30,2021-07-23,2021-07-16,2021-07-09,2021-07-02,2021-06-25,2021-06-18,2021-06-11,2021-06-04,2021-05-28,2021-05-21,2021-05-14,2021-05-07,2021-04-30,2021-04-23,2021-04-16,2021-04-09,2021-04-02,2021-03-26,2021-03-19,2021-03-12,2021-03-05,2021-02-26,2021-02-19,2021-02-12,2021-02-05,2021-01-29,2021-01-22,2021-01-15,2021-01-08,2021-01-01,2020-12-25,2020-12-18,2020-12-11,2020-12-04,2020-11-27,2020-11-20,2020-11-13,2020-11-06,2020-10-30,2020-10-23,2020-10-16,2020-10-09,2020-10-02,2020-09-25,2020-09-18,2020-09-11,2020-09-04,2020-08-28,2020-08-21,2020-08-14,2020-08-07,2020-07-31,2020-07-24,2020-07-17,2020-07-10,2020-07-03,2020-06-26,2020-06-19,2020-06-12,2020-06-05,2020-05-29,2020-05-22,2020-05-15,2020-05-08,2020-05-01,2020-04-24,2020-04-17,2020-04-10,2020-04-03,2020-03-27,2020-03-20,2020-03-13,2020-03-06,2020-02-28,2020-02-21,2020-02-14,2020-02-07,2020-01-31,2020-01-24,2020-01-17,2020-01-10,2020-01-03,2019-12-27,2019-12-20,2019-12-13,2019-12-06,2019-11-29,2019-11-22,2019-11-15,2019-11-08,2019-11-01,2019-10-25,2019-10-18,2019-10-11,2019-10-04,2019-09-27,2019-09-20,2019-09-13,2019-09-06,2019-08-30,2019-08-23,2019-08-16,2019-08-09,2019-08-02,2019-07-26,2019-07-19,2019-07-12,2019-07-05,2019-06-28,2019-06-21,2019-06-14,2019-06-07,2019-05-31,2019-05-24,2019-05-17,2019-05-10,2019-05-03,2019-04-26,2019-04-19,2019-04-12,2019-04-05,2019-03-29,2019-03-22,2019-03-15,2019-03-08,2019-03-01,2019-02-22,2019-02-15,2019-02-08,2019-02-01,2019-01-25,2019-01-18,2019-01-11,2019-01-04,2018-12-28,2018-12-21,2018-12-14,2018-12-07,2018-11-30,2018-11-23,2018-11-16,2018-11-09,2018-11-02,2018-10-26,2018-10-19,2018-10-12,2018-10-05,2018-09-28,2018-09-21,2018-09-14,2018-09-07,2018-08-31,2018-08-24,2018-08-17,2018-08-10,2018-08-03,2018-07-27,2018-07-20,2018-07-13,2018-07-06,2018-06-29,2018-06-22,2018-06-15,2018-06-08,2018-06-01,2018-05-25,2018-05-18,2018-05-11,2018-05-04,2018-04-27,2018-04-20,2018-04-13,2018-04-06,2018-03-30,2018-03-23,2018-03-16,2018-03-09,2018-03-02,2018-02-23,2018-02-16,2018-02-09,2018-02-02,2018-01-26,2018-01-19,2018-01-12,2018-01-05,2017-12-29,2017-12-15,2017-12-08,2017-12-01,2017-11-24,2017-11-17,2017-11-10,2017-11-03,2017-10-27,2017-10-20,2017-10-13,2017-10-06,2017-09-29,2017-09-22,2017-09-15,2017-09-08,2017-09-01,2017-08-25,2017-08-18,2017-08-11,2017-08-04,2017-07-28,2017-07-21,2017-07-14,2017-07-07,2017-06-30,2017-06-23,2017-06-16,2017-06-09,2017-06-02,2017-05-26,2017-05-19,2017-05-12,2017-05-05,2017-04-28,2017-04-21,2017-04-14,2017-04-07,2017-03-31,total_2022,total_2021,total_2020,total_2019,total_2018,total_2017,units,holiday_music
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Closer,The Chainsmokers,Closer,2016-09-09,5528407,0,5528407,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## Get song URIs from Spotipy

When searching up a song on Spotify, numerous versions of the same song can be returned. For this analysis (and for the creation of a playlist on another python document!), songs are taken from the explicit version of their parent album/single/EP, regardless of whether the song itself is explicit or clean. If no explicit version of the parent album/single/EP exists, the original clean version is to be used.

In order to utilize Spotify's API for the desired songs (from the Google Sheet), URIs must be retrieved using the following functions:

In [5]:
def search_for_URI(data):
    """
    Returns track URIs and the explicit status for a song

        Parameters:
            data (series): series containing a song title, artist, and the song's album
        
        Returns:
            song_info_list (list): list of song title, artist, album title, song URI and explicit status
                #otherwise, returns just the song title, artist, and album title
    """
    song_to_search = data['title'].replace("’","").replace("'","").lower()
    album_to_search = data['album'].replace("’","").replace("'","").lower()
    to_search = song_to_search + " " + data['artist']
    
    #this function searches for q on Spotify
    result = sp.search(q=to_search)
        
    #loops through all results from search
    for song_info_index in range(len(result['tracks']['items'])): 

        #checks for results' album to match the input data's album title (taking into account apostrophe formats)
        if result['tracks']['items'][song_info_index]['album']['name'].replace("’","").replace("'","").lower() == album_to_search:
                        
            #checks for results' song title to match input data's song title (taking into account apostrophe formats)
            if result['tracks']['items'][song_info_index]['name'].replace("’","").replace("'","").lower() == song_to_search:     
                
                #empty list to be filled with name/artist/album/explicit/uri info
                song_info_list = []

                #checks if a song is explicit       
                if result['tracks']['items'][song_info_index]['explicit'] == True:
                    song_info_list.append(result['tracks']['items'][song_info_index]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['artists'][0]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['album']['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['explicit'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['uri'])
                    return song_info_list
                
                #first, if a song is NOT explicit
                if result['tracks']['items'][song_info_index]['explicit'] == False:
                    song_info_list.append(result['tracks']['items'][song_info_index]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['artists'][0]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['album']['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['explicit'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['uri'])
                    return song_info_list
    
    #return just the song title/artist/album title if the URI is not found
    return [data['title'],data['artist'],data['album'],"",""]

In [6]:
#create a list of song info: song title, artist, album, explicit or not, track URI
uri_list = []
[uri_list.append(search_for_URI(gsheets_df.loc[song_info_index+1,'title':'album'])) for song_info_index in range(len(gsheets_df.loc[:]))]

#convert list to DataFrame
uri_df = pd.DataFrame(uri_list,columns=["song","artist","album","explicit","uri"])

#change index to start at 1
uri_df.index = range(1,len(uri_df)+1)
uri_df


,song,artist,album,explicit,uri
1,Closer,The Chainsmokers,Closer,False,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw
2,Fake Love,Drake,More Life,True,spotify:track:343YBumqHu19cGoGARUTsd
3,Starboy,The Weeknd,Starboy,True,spotify:track:7MXVkk9YMctZqd1Srtv4MB
4,Reminder,The Weeknd,Starboy,True,spotify:track:37F0uwRSrdzkBiuj0D5UHI
5,Both (feat. Drake),Gucci Mane,The Return of East Atlanta Santa,True,spotify:track:5tFep7dXGd7vEJ668wTPux
...,...,...,...,...,...
2148,Down Hill,Drake,"Honestly, Nevermind",False,spotify:track:3N3d4OWbPVxTQVBFqNyykk
2149,The Kind of Love We Make,Luke Combs,Growin' Up,False,spotify:track:48UKTR66uUOT9LaUvooTNx
2150,Tie That Binds,Drake,"Honestly, Nevermind",False,spotify:track:1XsRwo2it6QHdV2OiT6IzF
2151,U-Digg (feat. 42 Dugg & Veeze),Lil Baby,U-Digg (feat. 42 Dugg & Veeze),True,spotify:track:0b7UwotRCrS7qBoLDjfvSY


In [7]:
#create a dataframe with all songs that have no URI
data_to_clean = uri_df[uri_df['uri'] == ""]
data_to_clean.reset_index(drop=True, inplace=True)
data_to_clean

,song,artist,album,explicit,uri
0,Costa Rica,Dreamville,Revenge Of The Dreamers III,,
1,Levitating,Dua Lipa,Future Nostalgia (The Moonlight Edition),,


In [8]:
#check for any songs that don't have URIs
def missing_uri_detector(data_to_clean):
    """
    Returns a dataframe of songs with missing URIs if any exists

    Parameters:
        uri_df (DataFrame): the resulting dataframe from the search_for_URI function
    
    Returns
        data_to_clean (DataFrame): a subsection of uri_df with all songs with no URI
    """
    if len(data_to_clean) > 0 :
        if len(data_to_clean) == 1:
            print('There is 1 song with missing data.', 'Here is a dataframe with all songs with missing URIs:')
        else:
            print('There are', len(data_to_clean), 'songs with missing data.', 'Here is a dataframe with all songs with missing URIs:')
        return data_to_clean
    else:
        print('Success! The function successfully obtained the URIs for all songs')
              
missing_uri_detector(data_to_clean)


There are 2 songs with missing data. Here is a dataframe with all songs with missing URIs:


,song,artist,album,explicit,uri
0,Costa Rica,Dreamville,Revenge Of The Dreamers III,,
1,Levitating,Dua Lipa,Future Nostalgia (The Moonlight Edition),,


# Searching for missing URIs

Sometimes, not all URIs are found. Use the following functions to both search for missing URIs and input them into the dataframe

In [9]:
def song_uri_search(q):
    """
    Prints out different URIs (and other info) from a resulting search on spotipy
    
    Parameter:
        q (str): a string to look up using spotipy's search function
    """
    result = sp.search(q) 
    
    for song_info_dict in range(len(result['tracks']['items'])):
        print("Song: ", result['tracks']['items'][song_info_dict]['name'])
        print("Artist: ", result['tracks']['items'][song_info_dict]['artists'][0]['name'])
        print("Album: ", result['tracks']['items'][song_info_dict]['album']['name'])
        print("Explicit status: ", result['tracks']['items'][song_info_dict]['explicit'])
        print("URI: ", result['tracks']['items'][song_info_dict]['uri'],"\n")

In [10]:
def data_cleaning_func(data_to_clean, uri_df):
    #loop through every song in data_to_clean
    for i in range(0,len(data_to_clean)):
        print("Please follow the following steps to fill in the missing URI(s)." + "\n")
        song_to_search = data_to_clean.iloc[i]['song'].replace("’","").replace("'","")
        album_to_search = data_to_clean.iloc[i]['album'].replace("’","").replace("'","")
        artist_to_search = data_to_clean.iloc[i]['artist']
            
        #concatenate the song title, artist name, and album title to search
        to_search = song_to_search + " " + artist_to_search + " " + album_to_search

        result = sp.search(to_search)
        print("The following results are the search results from searching: '" + to_search + "'")
        print("\n")
            
        #call on song_uri_search function to print out search results
        song_uri_search(to_search)
        print("Search the URI(s) on your browser to double check the song is correct")
            
        #ensure user obtains desired result from the above search
        print("Did you obtain a desired result? Please type 'yes' or 'no': ")
        desired_result = str(input())
            
        #ensure desired_result is a valid input
        while (desired_result != 'yes')  and (desired_result != 'no'):
            print("Please give a valid input. Type either 'yes' or 'no': ")
            desired_result = str(input())
            
        #run the following in case the desired output is not available in the search above
        while desired_result == 'no':
            print("\nPlease manually search the information for this song (be as specific as possible): ")
            manual_search = str(input())
            song_uri_search(manual_search)
                
            #ask if another search is necessary
            print("\nWould you like to make another search? Please type in 'yes' or 'no': ")
            another_search = str(input())
                
            #ensure request for another search is valid
            while (another_search != 'yes')  and (another_search != 'no'):
                print("\nPlease give a valid input. Type either 'yes' or 'no': ")
                another_search = str(input())
                
            if another_search == "no":
                desired_result = 'yes'
            if another_search == "yes":
                desired_result = 'no'
            
        #the index of the song being searched, obtained from uri_df using song and album titles from data_to_clean
        missing_song_index = uri_df[(uri_df['song']==data_to_clean.iloc[i]['song']) & (uri_df['album']==data_to_clean.iloc[i]['album'])].index.values[0]
            
        #obtain explicit status of the song. Have user input the status because numerous songs can result from the search
        print("\nPlease copy and paste the explicit status of your selected song.")
        print("Alternatively, type in 'True' or 'False' (make sure the first character is capitalized): ")
        explicit_status = str(input())
            
        #check if explicit_status is valid
        while (explicit_status != "True")  and (explicit_status != "False"):
            print("\nPlease give a valid input. Type either 'True' or 'False': ")
            explicit_status = str(input())
            
        #to convert the string to a boolean
        explicit_status = eval(explicit_status)
            
        #obtain URI of the song. Have user input the URI because numerous songs can result from the search
        print("\nPlease copy and paste the entire URI for your selected song here: ")
        uri = str(input())
            
        #to give the user a chance to re-enter the URI in case they mispelled it
        print("\nDouble check that your URI is correct. Ready to confirm? Please enter 'yes' or 'no': ")
        confirmation = str(input())
            
        #check if confirmation is valid
        while (confirmation != 'yes')  and (confirmation != 'no'):
            print("\nPlease give a valid input. Type either 'yes' or 'no': ")
            confirmation = str(input())
            
        #ask for user to input URI again if they do not confirm they want to move foward. Continue asking after every input for uri
        if confirmation == 'no':
            print("\nPlease retype your URI here: ")
            uri = str(input())
                
            print("\nDouble check that your URI is correct. Ready to confirm? Please enter 'yes' or 'no': ")
            confirmation = str(input())
                
        #edit the uri dataframe!
        uri_df.loc[missing_song_index]['explicit'] = explicit_status
        uri_df.loc[missing_song_index]['uri'] = uri
            
        print("\nThe URI/explicit status for " + data_to_clean.loc[i]['song'] + " by " + artist_to_search + " is now updated on the uri_df!")
            
    #create a dataframe with all songs that have no URI
    data_to_clean = uri_df[uri_df['uri'] == ""]
    data_to_clean.reset_index(drop=True, inplace=True)
    
    print("\nAll URI/explicit statuses are accounted for!")

In [11]:
if len(data_to_clean) > 0:
    data_cleaning_func(data_to_clean, uri_df)

Please follow the following steps to fill in the missing URI(s).

The following results are the search results from searching: 'Costa Rica Dreamville Revenge Of The Dreamers III'


Song:  Costa Rica (with Bas & JID feat. Guapdad 4000, Reese LAFLARE, Jace, Mez, Smokepurpp, Buddy & Ski Mask The Slump God)
Artist:  Dreamville
Album:  Revenge Of The Dreamers III
Explicit status:  True
URI:  spotify:track:5WnfnSpuNEGXyEt78PBA6d 

Song:  Costa Rica (with Bas & JID feat. Guapdad 4000, Reese LAFLARE, Jace, Mez, Smokepurpp, Buddy & Ski Mask The Slump God)
Artist:  Dreamville
Album:  Revenge Of The Dreamers III: Director's Cut
Explicit status:  True
URI:  spotify:track:6stSUCyAY8mfBX00Nc5UuI 

Song:  Costa Rica (with Bas & JID feat. Guapdad 4000, Reese LAFLARE, Jace, Mez, Smokepurpp, Buddy, Ski Mask The Slump God)
Artist:  Dreamville
Album:  Revenge Of The Dreamers III
Explicit status:  False
URI:  spotify:track:3BgMbG1wEZlb9w4OqPFncq 

Song:  Costa Rica (with Bas & JID feat. Guapdad 4000, Reese

In [12]:
print("Success! The entire dataframe is now updated with song URIs! Please wait for Spotify's track features to be retreived.")

Success! The entire dataframe is now updated with song URIs! Please wait for Spotify's track features to be retreived.


## Get spotipy's API Information

Spotify has a plethora of information on a song's characteristics. The following functions/cells create a dataframe with song characteristic information on each song from the Google Sheet. Information of what each variable means can be found here: https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features

In [13]:
def getTrackFeatures(track_id):
    """
    Returns a list of selected information from Spotify's API given a song's URI

    Parameters:
        track_id (str): a track's URI
    
    Returns
        track (list): a list of song information including it's danceability, length, tempo, etc. 
    """
    md = sp.track(track_id)
    features = sp.audio_features(track_id)
    
    #meta data
    name = md['name']
    album = md['album']['name']
    artist = md['album']['artists'][0]['name']
    release_date = md['album']['release_date']
    length = md['duration_ms']
    popularity = md['popularity']
    
    #features from the data
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    key = features[0]['key']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    mode = features[0]['mode']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']
    valence = features[0]['valence']
    
    #putting it all together
    track = [name, album, artist, release_date, length, popularity, 
             acousticness, danceability, energy, instrumentalness, key,
            liveness, loudness, mode, speechiness, 
             tempo, time_signature, valence]
    return track

In [14]:
#Get track features for all tracks, append to a list
track_features = []
[track_features.append( getTrackFeatures(uri_df['uri'][i]) ) for i in range(1,len(uri_df['uri'])+1) if (uri_df['uri'][i] != "")]

track_features

[['Closer',
  'Closer',
  'The Chainsmokers',
  '2016-07-29',
  244960,
  85,
  0.414,
  0.748,
  0.524,
  0,
  8,
  0.111,
  -5.599,
  1,
  0.0338,
  95.01,
  4,
  0.661],
 ['Fake Love',
  'More Life',
  'Drake',
  '2017-03-18',
  210937,
  75,
  0.105,
  0.928,
  0.481,
  0,
  9,
  0.176,
  -9.35,
  0,
  0.287,
  134.007,
  4,
  0.613],
 ['Starboy',
  'Starboy',
  'The Weeknd',
  '2016-11-25',
  230453,
  86,
  0.141,
  0.679,
  0.587,
  6.35e-06,
  7,
  0.137,
  -7.015,
  1,
  0.276,
  186.003,
  4,
  0.486],
 ['Reminder',
  'Starboy',
  'The Weeknd',
  '2016-11-25',
  218880,
  74,
  0.142,
  0.706,
  0.505,
  0,
  8,
  0.164,
  -6.923,
  1,
  0.193,
  160.053,
  4,
  0.388],
 ['Both (feat. Drake)',
  'The Return of East Atlanta Santa',
  'Gucci Mane',
  '2016-12-16',
  190311,
  70,
  0.119,
  0.85,
  0.405,
  0.000118,
  7,
  0.0707,
  -7.509,
  0,
  0.225,
  139.976,
  4,
  0.344],
 ['Paris',
  'Memories...Do Not Open',
  'The Chainsmokers',
  '2017-04-07',
  221506,
  74,
  0.0

In [15]:
#Convert list to DataFrame
api_info_df = pd.DataFrame(track_features,columns=['name','album','artist',
    'release_date','length','popularity','acousticness','danceability',
    'energy','instrumentalness','key','liveness','loudness','mode',
    'speechiness','tempo','time_signature','valence'])

#change index to start at one
api_info_df.index = range(1,len(api_info_df)+1)

#change release_date to date format
api_info_df['release_date'] = pd.to_datetime(api_info_df['release_date'])

api_info_df

,name,album,artist,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
1,Closer,Closer,The Chainsmokers,2016-07-29,244960,85,0.414000,0.748,0.524,0.000000,8,0.1110,-5.599,1,0.0338,95.010,4,0.661
2,Fake Love,More Life,Drake,2017-03-18,210937,75,0.105000,0.928,0.481,0.000000,9,0.1760,-9.350,0,0.2870,134.007,4,0.613
3,Starboy,Starboy,The Weeknd,2016-11-25,230453,86,0.141000,0.679,0.587,0.000006,7,0.1370,-7.015,1,0.2760,186.003,4,0.486
4,Reminder,Starboy,The Weeknd,2016-11-25,218880,74,0.142000,0.706,0.505,0.000000,8,0.1640,-6.923,1,0.1930,160.053,4,0.388
5,Both (feat. Drake),The Return of East Atlanta Santa,Gucci Mane,2016-12-16,190311,70,0.119000,0.850,0.405,0.000118,7,0.0707,-7.509,0,0.2250,139.976,4,0.344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2148,Down Hill,"Honestly, Nevermind",Drake,2022-06-17,250864,79,0.965000,0.718,0.319,0.000360,3,0.0895,-11.261,1,0.0435,109.994,4,0.639
2149,The Kind of Love We Make,Growin' Up,Luke Combs,2022-06-24,224426,68,0.022600,0.608,0.745,0.000006,1,0.0942,-4.130,1,0.0277,102.025,4,0.464
2150,Tie That Binds,"Honestly, Nevermind",Drake,2022-06-17,336005,79,0.017600,0.775,0.688,0.091700,4,0.1080,-10.912,0,0.0593,120.017,4,0.284
2151,U-Digg (feat. 42 Dugg & Veeze),U-Digg (feat. 42 Dugg & Veeze),Lil Baby,2022-06-17,235500,75,0.002530,0.824,0.583,0.000000,1,0.2660,-6.537,1,0.1470,147.496,4,0.154


## Joining all the data

Combine all the data collected/generated above!

In [16]:
#concatinate all variables into a final dataframe
final_df = pd.concat([gsheets_df.loc[:,:], uri_df.loc[:,'uri'], api_info_df.loc[:,'release_date':'valence'], uri_df.loc[:,'explicit']]
            ,axis=1)

#reset index and convert all integers/float/dates to strings in preparation to convert to a Google Shet
final_df = final_df.reset_index()

final_df

,index,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,peak_week_streams,current_year,2022-06-17,2022-06-10,2022-06-03,2022-05-27,2022-05-20,2022-05-13,2022-05-06,2022-04-29,2022-04-22,2022-04-15,2022-04-08,2022-04-01,2022-03-25,2022-03-18,2022-03-11,2022-03-04,2022-02-25,2022-02-18,2022-02-11,2022-02-04,2022-01-28,2022-01-21,2022-01-14,2022-01-07,2021-12-31,2021-12-24,2021-12-17,2021-12-10,2021-12-03,2021-11-26,2021-11-19,2021-11-12,2021-11-05,2021-10-29,2021-10-22,2021-10-15,2021-10-08,2021-10-01,2021-09-24,2021-09-17,2021-09-10,2021-09-03,2021-08-27,2021-08-20,2021-08-13,2021-08-06,2021-07-30,2021-07-23,2021-07-16,2021-07-09,2021-07-02,2021-06-25,2021-06-18,2021-06-11,2021-06-04,2021-05-28,2021-05-21,2021-05-14,2021-05-07,2021-04-30,2021-04-23,2021-04-16,2021-04-09,2021-04-02,2021-03-26,2021-03-19,2021-03-12,2021-03-05,2021-02-26,2021-02-19,2021-02-12,2021-02-05,2021-01-29,2021-01-22,2021-01-15,2021-01-08,2021-01-01,2020-12-25,2020-12-18,2020-12-11,2020-12-04,2020-11-27,2020-11-20,2020-11-13,2020-11-06,2020-10-30,2020-10-23,2020-10-16,2020-10-09,2020-10-02,2020-09-25,2020-09-18,2020-09-11,2020-09-04,2020-08-28,2020-08-21,2020-08-14,2020-08-07,2020-07-31,2020-07-24,2020-07-17,2020-07-10,2020-07-03,2020-06-26,2020-06-19,2020-06-12,2020-06-05,2020-05-29,2020-05-22,2020-05-15,2020-05-08,2020-05-01,2020-04-24,2020-04-17,2020-04-10,2020-04-03,2020-03-27,2020-03-20,2020-03-13,2020-03-06,2020-02-28,2020-02-21,2020-02-14,2020-02-07,2020-01-31,2020-01-24,2020-01-17,2020-01-10,2020-01-03,2019-12-27,2019-12-20,2019-12-13,2019-12-06,2019-11-29,2019-11-22,2019-11-15,2019-11-08,2019-11-01,2019-10-25,2019-10-18,2019-10-11,2019-10-04,2019-09-27,2019-09-20,2019-09-13,2019-09-06,2019-08-30,2019-08-23,2019-08-16,2019-08-09,2019-08-02,2019-07-26,2019-07-19,2019-07-12,2019-07-05,2019-06-28,2019-06-21,2019-06-14,2019-06-07,2019-05-31,2019-05-24,2019-05-17,2019-05-10,2019-05-03,2019-04-26,2019-04-19,2019-04-12,2019-04-05,2019-03-29,2019-03-22,2019-03-15,2019-03-08,2019-03-01,2019-02-22,2019-02-15,2019-02-08,2019-02-01,2019-01-25,2019-01-18,2019-01-11,2019-01-04,2018-12-28,2018-12-21,2018-12-14,2018-12-07,2018-11-30,2018-11-23,2018-11-16,2018-11-09,2018-11-02,2018-10-26,2018-10-19,2018-10-12,2018-10-05,2018-09-28,2018-09-21,2018-09-14,2018-09-07,2018-08-31,2018-08-24,2018-08-17,2018-08-10,2018-08-03,2018-07-27,2018-07-20,2018-07-13,2018-07-06,2018-06-29,2018-06-22,2018-06-15,2018-06-08,2018-06-01,2018-05-25,2018-05-18,2018-05-11,2018-05-04,2018-04-27,2018-04-20,2018-04-13,2018-04-06,2018-03-30,2018-03-23,2018-03-16,2018-03-09,2018-03-02,2018-02-23,2018-02-16,2018-02-09,2018-02-02,2018-01-26,2018-01-19,2018-01-12,2018-01-05,2017-12-29,2017-12-15,2017-12-08,2017-12-01,2017-11-24,2017-11-17,2017-11-10,2017-11-03,2017-10-27,2017-10-20,2017-10-13,2017-10-06,2017-09-29,2017-09-22,2017-09-15,2017-09-08,2017-09-01,2017-08-25,2017-08-18,2017-08-11,2017-08-04,2017-07-28,2017-07-21,2017-07-14,2017-07-07,2017-06-30,2017-06-23,2017-06-16,2017-06-09,2017-06-02,2017-05-26,2017-05-19,2017-05-12,2017-05-05,2017-04-28,2017-04-21,2017-04-14,2017-04-07,2017-03-31,total_2022,total_2021,total_2020,total_2019,total_2018,total_2017,units,holiday_music,uri,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,explicit
0,1,Closer,The Chainsmokers,Closer,2016-09-09,5528407,0,5528407,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5528407,5420007,0,0,0,0,0,10948414,36856,0,

## Converting the Dataframe to a Google Sheet

The codes above take a long time to run! To ease the data collection process and save time for research purposes, the dataframe is to be converted to a Google Sheet (same link with the HITS Streaming data) using the following code:

In [17]:
url = "https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit#gid=0"

spreadsheet_key = url.split("/")[-2]

wks_name = 'DF to Gspread'

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('gspread_service_account.json', scope)

gc = gspread.authorize(credentials)

d2g.upload(final_df, spreadsheet_key, wks_name, credentials = credentials, row_names=False)

<Worksheet 'DF to Gspread' id:2071085388>

You can now use the worksheet above for analytical purposes!! Happy analyzing! :)